# Calculating Protected areas at the Ethnologue Polygon Level

In [3]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties

import mapclassify

from rapidfuzz import process, fuzz

from shapely.geometry import Point
from shapely.geometry import MultiPolygon

import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterstats import zonal_stats
from glob import glob
from rasterio.merge import merge
from rasterio.enums import Resampling
from rasterio.io import MemoryFile

In [4]:
# Set base project path
base_path = Path("C:/Users/juami/Dropbox/RAships/2-Folklore-Nathan-Project/EA-Maps-Nathan-project/Measures_work")

# Set file paths
poscol_path = base_path / "data" / "raw" / "ethnologue" / "ancestral_characteristics_database_language_level" / "Ethnologue_16_shapefile" / "langa_no_overlap_biggest_clean.shp"

data_path = base_path / "data" / "interim"
maps_path = base_path / "maps" / "raw"
protectedland_path = maps_path / "Protected_land"

In [6]:
# Read the shapefiles and CSV files
ethnologue = gpd.read_file(poscol_path)
ethnologue = ethnologue.to_crs(epsg=6933)
ethnologue = ethnologue[["ID", "geometry"]]

protected0 = gpd.read_file(protectedland_path / "WDPA_Jun2025_Public_shp_0" / "WDPA_Jun2025_Public_shp-polygons.shp")
protected1 = gpd.read_file(protectedland_path / "WDPA_Jun2025_Public_shp_1" / "WDPA_Jun2025_Public_shp-polygons.shp")   
protected2 = gpd.read_file(protectedland_path / "WDPA_Jun2025_Public_shp_2" / "WDPA_Jun2025_Public_shp-polygons.shp")



In [7]:
# Combine the three GeoDataFrames
protected_all = pd.concat([protected0, protected1, protected2], ignore_index=True)

# Ensure it's still a GeoDataFrame
protected_all = gpd.GeoDataFrame(protected_all, geometry='geometry', crs=protected0.crs)
protected_all = protected_all.to_crs(epsg=6933)

In [8]:
protected_all=protected_all[['WDPAID', 'geometry']]
protected_all.head()

,WDPAID,geometry
0,1.0,"POLYGON ((-5.97e+06 2.16e+06, -5.97e+06 2.16e+..."
1,2.0,"POLYGON ((-5.96e+06 2.2e+06, -5.96e+06 2.2e+06..."
2,3.0,"POLYGON ((-6.37e+06 -2.8e+06, -6.37e+06 -2.8e+..."
3,4.0,"POLYGON ((-5.97e+06 -3e+06, -5.97e+06 -3e+06, ..."
4,6.0,"POLYGON ((-7.06e+06 -5.55e+06, -7.06e+06 -5.55..."


In [9]:
# Making sure same CRS
ethnologue = ethnologue.to_crs(protected_all.crs)

# Keeping intersecting geometries 
intersections = gpd.overlay(ethnologue, protected_all, how='intersection')

# Calculate the area of the intersections in square kilometers
intersections['protected_km2'] = intersections.geometry.area / 1e6

# Group intersections by Ethnologue ID and sum the protected areas
protected_by_ethno = intersections.groupby('ID', as_index=False)['protected_km2'].sum()

#Merging to original
ethnologue_protectedland = ethnologue.merge(protected_by_ethno, on='ID', how='left')
ethnologue_protectedland['protected_km2'] = ethnologue_protectedland['protected_km2'].fillna(0)

C:\Users\juami\AppData\Local\Temp\ipykernel_3348\765946773.py:5: UserWarning: `keep_geom_type=True` in overlay resulted in 70 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersections = gpd.overlay(ethnologue, protected_all, how='intersection')


In [10]:
ethnologue_protectedland.head()

,ID,geometry,protected_km2
0,RUS-RUS,"MULTIPOLYGON (((4.35e+06 5.01e+06, 4.35e+06 5....",6.923853e+05
1,ENG-USA,"MULTIPOLYGON (((-7.25e+06 2.5e+06, -7.25e+06 2...",7.824268e+05
2,POR-BRA,"MULTIPOLYGON (((-5.03e+06 -3.88e+06, -5.03e+06...",1.824773e+06
3,ENG-AUS,"MULTIPOLYGON (((1.42e+07 -5e+06, 1.42e+07 -5e+...",5.459749e+05
4,CMN-CHN,"MULTIPOLYGON (((1.05e+07 2.39e+06, 1.05e+07 2....",3.524290e+04


In [11]:
# Keep only relevant columns
ethnologue_protectedarea = ethnologue_protectedland[["ID", "protected_km2"]]

# Export to CSV
ethnologue_protectedarea.to_csv(protectedland_path / "ethnologue_wdpa.csv", index=False)

print(f"Exported ethnologue_wdpa.csv")

Exported ethnologue_wdpa.csv
